In [1]:
device="cpu"
import io
import torch
import torch.nn as nn
import PIL.Image
import einops
import matplotlib.pyplot as plt
import numpy as np
import datasets
import math
from IPython.display import HTML
from types import SimpleNamespace
from timm.optim import Mars
from fastprogress import progress_bar, master_bar
from torchvision.transforms.v2 import ToPILImage, PILToTensor, CenterCrop, RandomCrop
from timm.models.efficientvit_mit import GELUTanh, EfficientVitBlock, ResidualBlock, build_local_block
from pytorch_wavelets import DWTForward, DWTInverse
from tft.wavelet import WPT3D, IWPT3D
from tft.utils import compand, decompand
dataset = datasets.load_dataset("danjacobellis/organmnist3d_64")

In [2]:
config = SimpleNamespace()
config.F = 8
config.target_bits_per_channel = np.log2(7)
config.latent_dim = 128;
config.channels = 1
config.max_noise_penalty = 1e-5
config.encoder_depth = 6
config.max_lr = 1e-3
config.min_lr = config.max_lr / 1e3
config.lr_pow = 2
config.volume_size = 48
config.batch_size = 64
config.num_workers = 32
config.epochs = 24
config.total_steps = config.epochs * (dataset['train'].num_rows // config.batch_size)

In [3]:
from transforms import RandomCrop3D
rand_crop = RandomCrop3D(64)
def pil_to_grid3d(img):
    x = torch.tensor(np.array(img))
    x = einops.rearrange(x, '(a y) (b z) c -> (a b c) y z', a=4, b=4, c=4)
    return x
def collate_fn(batch):
    return torch.cat([
        rand_crop(pil_to_grid3d(sample['image']).unsqueeze(0).unsqueeze(0)) for sample in batch
    ]).to(torch.float)/127.5 - 1.0

In [5]:
batch = dataset['train'].select(range(8))
x = collate_fn(batch)

In [7]:
from IPython.display import display, clear_output
from ipywidgets import interact

volume = x[3,0].numpy()
def show_slice(index):
    clear_output(wait=True)  # Clear previous output
    slice_data = volume[index, :, :]
    img = ToPILImage()(slice_data/2+0.5).resize((256,256),resample=PIL.Image.Resampling.LANCZOS)
    display(img)
interact(show_slice, index=(0, 47, 1));

interactive(children=(IntSlider(value=23, description='index', max=47), Output()), _dom_classes=('widget-inter…